# 参数管理

一旦我们选择了架构并设置了超参数，我们就进入训练循环，在这个循环中我们的目标是找到使损失函数最小化的参数值。训练完成后，为了进行未来的预测，我们需要这些参数。此外，有时我们希望提取参数，可能是为了在其他上下文中重用它们，将模型保存到磁盘以便在其他软件中执行，或者为了通过检查来获得科学理解。

大多数时候，我们可以忽略参数声明和操作的具体细节，依靠深度学习框架来完成繁重的工作。然而，当我们远离具有标准层的堆叠架构时，有时需要深入了解参数的声明和操作。在本节中，我们将涵盖以下内容：

* 为调试、诊断和可视化访问参数。
* 在不同的模型组件之间共享参数。

In [1]:
import torch
from torch import nn

（**我们首先关注一个含有一层隐藏层的多层感知机。**）

In [2]:
net = nn.Sequential(nn.LazyLinear(8),
                    nn.ReLU(),
                    nn.LazyLinear(1))

X = torch.rand(size=(2, 4))
net(X).shape

torch.Size([2, 1])

## [**参数访问**]
:label:`subsec_param-access`

让我们从如何访问你已知模型中的参数开始。

当通过`Sequential`类定义模型时，我们可以通过像列表一样索引模型来访问任何层。每个层的参数都方便地存储在其属性中。

我们可以检查第二个全连接层的参数如下。

In [3]:
net[2].state_dict()

OrderedDict([('weight',
              tensor([[ 0.1097, -0.2902, -0.2685,  0.0790, -0.1595,  0.0364,  0.2286,  0.0852]])),
             ('bias', tensor([-0.2319]))])

我们可以看到这个全连接层包含两个参数，分别对应该层的权重和偏置。

### [**目标参数**]

请注意，每个参数都表示为参数类的一个实例。要对这些参数进行任何有用的操作，我们首先需要访问其底层的数值。有几种方法可以做到这一点，有些方法更简单，而有些则更为通用。以下代码从第二神经网络层中提取偏置，这将返回一个参数类实例，并进一步访问该参数的值。

In [4]:
type(net[2].bias), net[2].bias.data

(torch.nn.parameter.Parameter, tensor([-0.2319]))

参数是复杂的对象，
包含值、梯度
以及额外的信息。
这就是为什么我们需要显式地请求值。

除了值之外，每个参数还允许我们访问梯度。因为我们还没有为这个网络调用反向传播，所以它处于初始状态。

In [5]:
net[2].weight.grad == None

True

### [**一次性处理所有参数**]

当我们需要对所有参数执行操作时，
逐一访问它们可能会变得很繁琐。
特别是在处理更复杂的情况，例如嵌套模块时，
情况会变得更加难以管理，
因为我们需要递归遍历整个树来提取
每个子模块的参数。下面我们将演示如何访问所有层的参数。

In [6]:
[(name, param.shape) for name, param in net.named_parameters()]

[('0.weight', torch.Size([8, 4])),
 ('0.bias', torch.Size([8])),
 ('2.weight', torch.Size([1, 8])),
 ('2.bias', torch.Size([1]))]

## [**Tied Parameters**]

通常，我们希望在多个层之间共享参数。
让我们看看如何优雅地实现这一点。
在下面的例子中，我们分配了一个全连接层，
然后专门使用它的参数来设置另一个层的参数。
这里我们需要运行前向传播
`net(X)` 才能访问参数。

In [7]:
# We need to give the shared layer a name so that we can refer to its
# parameters
shared = nn.LazyLinear(8)
net = nn.Sequential(nn.LazyLinear(8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.LazyLinear(1))

net(X)
# Check whether the parameters are the same
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# Make sure that they are actually the same object rather than just having the
# same value
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


这个例子表明第二层和第三层的参数是绑定的。它们不仅仅是相等，而是由完全相同的张量表示。因此，如果我们更改其中一个参数，另一个也会随之改变。

你可能想知道，
当参数被绑定时
梯度会发生什么？
由于模型参数包含梯度，
在反向传播过程中
第二隐藏层和第三隐藏层的梯度会被加在一起。

## 摘要

我们有多种方法可以访问和绑定模型参数。


## 练习

1. 使用在 :numref:`sec_model_construction` 中定义的 `NestMLP` 模型，访问各层的参数。
1. 构建一个包含共享参数层的多层感知机并进行训练。在训练过程中，观察每一层的模型参数和梯度。
1. 为什么共享参数是一个好主意？

[讨论](https://discuss.d2l.ai/t/57)